In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. Name of the environmental variables is arbitraray because langrila modules accepts that name as an argument.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In [3]:
from langrila import Agent, Tool
from langrila.anthropic import AnthropicClient
from langrila.aws import BedrockClient
from langrila.google import GoogleClient
from langrila.openai import OpenAIClient

# Define tools

In this example, we use dummy tools as shown below. In langrila, tool's schema is generated from the tool's type hint and docstring in default, so it's important to write in details. Here is an example using dummy tools.

Tool can have a pydantic model in its arguments. Nested pydantic model is also acceptable.

In [4]:
import random
from enum import Enum

from pydantic import BaseModel, Field


class MusicGenre(str, Enum):
    rock = "rock"
    pop = "pop"
    jazz = "jazz"
    classical = "classical"
    hip_hop = "hip-hop"


class EqualizerSettings(BaseModel):
    hz_100: int = Field(0, ge=-12, le=12, description="The 100 Hz band in dB.")
    hz_200: int = Field(0, ge=-12, le=12, description="The 200 Hz band in dB.")
    hz_400: int = Field(0, ge=-12, le=12, description="The 400 Hz band in dB.")
    hz_800: int = Field(0, ge=-12, le=12, description="The 800 Hz band in dB.")
    hz_1600: int = Field(0, ge=-12, le=12, description="The 1600 Hz band in dB.")
    hz_3200: int = Field(0, ge=-12, le=12, description="The 3200 Hz band in dB.")
    hz_6400: int = Field(0, ge=-12, le=12, description="The 4000 Hz band in dB.")


def start_music(genre: MusicGenre) -> str:
    """
    Start playing a random music with the given the genre.

    Parameters
    ----------
    genre : MusicGenre
        The genre of the music to play.

    Returns
    ----------
    str
        A message indicating that the music has started.
    """

    bpms = {
        MusicGenre.rock: 150,
        MusicGenre.pop: 110,
        MusicGenre.jazz: 120,
        MusicGenre.classical: 76,
        MusicGenre.hip_hop: 100,
    }

    return (
        f"Starting music! Genre: {genre}, "
        f"Volume: {random.uniform(0.0, 1.0)}, "
        f"BPM: {bpms[genre]}"
    )


def change_music_volume(volume: float) -> str:
    """
    Change the music volume.

    Parameters
    ----------
    volume : float
        The new volume to set. It should be between 0.0 and 1.0.

    Returns
    ----------
    str
        A message indicating that the volume has been changed.
    """
    return f"Changing volume to {volume}"


def change_music(genre: MusicGenre) -> str:
    """
    Change the music.

    Parameters
    ----------
    genre : MusicGenre
        The genre of the music to change to.

    Returns
    ----------
    str
        A message indicating that the music has been changed.
    """
    bpm_ranges = {
        MusicGenre.rock: (120, 180),
        MusicGenre.pop: (80, 140),
        MusicGenre.jazz: (70, 120),
        MusicGenre.classical: (70, 100),
        MusicGenre.hip_hop: (80, 120),
    }

    new_bpm = random.randint(bpm_ranges[genre][0], bpm_ranges[genre][1])

    return f"Turning to another music! Genre: {genre}, BPM: {new_bpm}"


def change_equalizer_settings(settings: EqualizerSettings) -> str:
    """
    Change the equalizer settings of the music.

    Parameters
    ----------
    settings : EqualizerSettings
        The new equalizer settings to set.

    Returns
    ----------
    str
        A message indicating that the equalizer settings have been changed.
    """
    return f"Changing equalizer settings to {settings.model_dump(exclude_none=True, exclude_unset=True)}"

# Instantiating Agent

Client modules

In [5]:
# For OpenAI
openai_client = OpenAIClient(api_key_env_name="OPENAI_API_KEY")

# For Azure OpenAI
azure_openai_client = OpenAIClient(
    api_key_env_name="AZURE_API_KEY",
    api_type="azure",
    azure_api_version="2024-11-01-preview",
    azure_endpoint_env_name="AZURE_ENDPOINT",
    azure_deployment_env_name="AZURE_DEPLOYMENT_ID",
)

# For Gemini on Google AI Studio
google_dev_client = GoogleClient(
    api_key_env_name="GEMINI_API_KEY",
)

# For Gemini on Google Cloud VertexAI
vertexai_client = GoogleClient(
    api_type="vertexai",
    project_id_env_name="GOOGLE_CLOUD_PROJECT",
    location="us-central1",
)

# For Claude of Anthropic
anthropic_client = AnthropicClient(
    api_key_env_name="ANTHROPIC_API_KEY",
)

# For Claude of Amazon Bedrock
claude_bedrock_client = AnthropicClient(
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region="us-east-1",
)

bedrock_client = BedrockClient(
    region_name="us-east-1",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
)

Tools must be specified when the agent is initialized.

In [6]:
tools = [start_music, change_music_volume, change_music, change_equalizer_settings]

In [7]:
openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
)

# Generating text with tool calling

In [8]:
prompt = "Play some music, well, play jazz music and prefer to low volume."

## Synchronous, non-streaming text generation

In [9]:
response = openai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-14 10:01:48] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:01:48] INFO | root: Generating text
[2025-01-14 10:01:50] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_TyHpbVnPPbGNZjDBJcAXSkX4'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='call_klWS0vSxbXDK3SprGwRWMWFK')]
[2025-01-14 10:01:50] INFO | Running tool: start_music
[2025-01-14 10:01:50] INFO | Tool: start_music successfully ran.
[2025-01-14 10:01:50] INFO | Running tool: change_music_volume
[2025-01-14 10:01:50] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:01:50] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.04480601438948806, BPM: 120', error=None, call_id='call_TyHpbVnPPbGNZjDBJcAXSkX4', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='call_klWS0vSxb

I've started playing some jazz music at a low volume. Enjoy the tunes!


Agent finally generates text response. Steps the agent generates final answer is like: 

1. Feed user's input
2. Call tools
3. Validate args with the pydantic schema validator
4. Actually run called tools
5. Generate text response based on the tool calling results

You can specify `planning` argument when the agent is instantiated. If `planning` is True, the agent makes a plan to answer the user's input based on the given tools, and then generates answer.

In [10]:
openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
    planning=True,  # <- Enable planning
)

In [11]:
response = openai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-14 10:01:51] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, play jazz music and prefer to low volume.\n\nCapabilities:\n- start_music: Start playing a random music with the given the genre.\n- change_music_volume: Change the music volume.\n- change_music: Change the music.\n- change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-14 10:01:51] INFO | root: Generating text
[2025-01-14 10:01:53] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Playing Jazz Music**: Use the `start_music` tool to begin playing jazz music.\n2. **Set Volume to Low**: Use the `change_music_volume` tool to adjust the vo

Jazz music has started playing, and the volume has been set to a low level. Enjoy the music!


By planning, `start_music` tool was called at the first step, then `change_bpm ` tool and `change_music` tool was.

Here is the default planning prompt.

In [12]:
from langrila import AgentConfig

print(AgentConfig().internal_prompt.planning)

Please make a concise plan to answer the following question/requirement, considering the conversation history.
You can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.
Agent has no description while the tools have a description.

Question/Requirement:
{user_input}

Capabilities:
{capabilities}


This AgentConfig is configurable and customized config can be passes to the agent when instantiating as mentioned in [01.introduction.ipynb](./01.introduction.ipynb).

## Asynchronous, non-streaming text generation

In [13]:
response = await openai_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-14 10:01:55] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, play jazz music and prefer to low volume.\n\nCapabilities:\n- start_music: Start playing a random music with the given the genre.\n- change_music_volume: Change the music volume.\n- change_music: Change the music.\n- change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-14 10:01:55] INFO | root: Generating text
[2025-01-14 10:01:57] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Playing Jazz Music**: Use the `start_music` tool to begin playing jazz music.\n2. **Set Volume to Low**: Use the `change_music_volume` tool to adjust the vo

Jazz music has started playing, and the volume has been set to a low level. Enjoy the music!


## Synchronous, streaming text generation

In [14]:
from langrila import TextResponse, ToolCallResponse

In [15]:
streamed_response = openai_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:01:59] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, play jazz music and prefer to low volume.\n\nCapabilities:\n- start_music: Start playing a random music with the given the genre.\n- change_music_volume: Change the music volume.\n- change_music: Change the music.\n- change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-14 10:01:59] INFO | root: Generating text
[2025-01-14 10:02:01] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Playing Jazz Music**: Use the `start_music` tool to begin playing jazz music.\n2. **Set Volume to Low**: Use the `change_music_volume` tool to adjust the vo

{"ge
{"genre":
{"genre": "jazz
{"genre": "jazz"}
{"vo
{"volume"
{"volume": 0.2}


[2025-01-14 10:02:02] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_v9ij45yjlQaXvH4Fo7VRa1Fx'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='call_EgypEw1WxRmXTdnANOEEUgpn')]
[2025-01-14 10:02:02] INFO | Running tool: start_music
[2025-01-14 10:02:02] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:02] INFO | Running tool: change_music_volume
[2025-01-14 10:02:02] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:02] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.4757927049110693, BPM: 120', error=None, call_id='call_v9ij45yjlQaXvH4Fo7VRa1Fx', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='call_EgypEw1WxRmXTdnANOEEUgpn', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-14 10:02:02] INFO | root: Generating text


Jazz
Jazz music
Jazz music has
Jazz music has started
Jazz music has started playing
Jazz music has started playing,
Jazz music has started playing, and
Jazz music has started playing, and the
Jazz music has started playing, and the volume
Jazz music has started playing, and the volume has
Jazz music has started playing, and the volume has been
Jazz music has started playing, and the volume has been set
Jazz music has started playing, and the volume has been set to
Jazz music has started playing, and the volume has been set to a
Jazz music has started playing, and the volume has been set to a low
Jazz music has started playing, and the volume has been set to a low level
Jazz music has started playing, and the volume has been set to a low level.
Jazz music has started playing, and the volume has been set to a low level. Enjoy
Jazz music has started playing, and the volume has been set to a low level. Enjoy the
Jazz music has started playing, and the volume has been set to a low level. E

[2025-01-14 10:02:03] DEBUG | Response: [TextResponse(text='Jazz music has started playing, and the volume has been set to a low level. Enjoy the music!')]


Jazz music has started playing, and the volume has been set to a low level. Enjoy the music!


In [16]:
streamed_response = openai_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:02:03] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, play jazz music and prefer to low volume.\n\nCapabilities:\n- start_music: Start playing a random music with the given the genre.\n- change_music_volume: Change the music volume.\n- change_music: Change the music.\n- change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-14 10:02:03] INFO | root: Generating text
[2025-01-14 10:02:05] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Playing Jazz Music**: Use the `start_music` tool to begin playing jazz music.\n2. **Set Volume to Low**: Use the `change_music_volume` tool to adjust the vo

{"ge
{"genre":
{"genre": "jazz
{"genre": "jazz"}
{"vo
{"volume"
{"volume": 0.2}


[2025-01-14 10:02:06] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_BX963mFrJ9J4WtVOipVTv4WZ'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='call_T05O74psYWk3AJNxs9W1RxCB')]
[2025-01-14 10:02:06] INFO | Running tool: start_music
[2025-01-14 10:02:07] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:07] INFO | Running tool: change_music_volume
[2025-01-14 10:02:07] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:07] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.36418591620466745, BPM: 120', error=None, call_id='call_BX963mFrJ9J4WtVOipVTv4WZ', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='call_T05O74psYWk3AJNxs9W1RxCB', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-14 10:02:07] INFO | root: Generating text


Jazz
Jazz music
Jazz music has
Jazz music has started
Jazz music has started playing
Jazz music has started playing,
Jazz music has started playing, and
Jazz music has started playing, and the
Jazz music has started playing, and the volume
Jazz music has started playing, and the volume has
Jazz music has started playing, and the volume has been
Jazz music has started playing, and the volume has been set
Jazz music has started playing, and the volume has been set to
Jazz music has started playing, and the volume has been set to a
Jazz music has started playing, and the volume has been set to a low
Jazz music has started playing, and the volume has been set to a low level
Jazz music has started playing, and the volume has been set to a low level.
Jazz music has started playing, and the volume has been set to a low level. Enjoy
Jazz music has started playing, and the volume has been set to a low level. Enjoy the
Jazz music has started playing, and the volume has been set to a low level. E

[2025-01-14 10:02:07] DEBUG | Response: [TextResponse(text='Jazz music has started playing, and the volume has been set to a low level. Enjoy the music!')]


Jazz music has started playing, and the volume has been set to a low level. Enjoy the music!


## Other client

Other clients has the same interface.

In [17]:
# Azure OpenAI
azure_openai_agent = Agent(
    client=azure_openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
    # planning=True, # as needed
)

# Google AI Studio
google_agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=tools,
    # planning=True, # as needed
)

# Google Cloud VertexAI
vertexai_agent = Agent(
    client=vertexai_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=tools,
    # planning=True, # as needed
)

# Anthropic Claude
claude_agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    temperature=0.0,
    max_tokens=500,
    tools=tools,
    # planning=True, # as needed
)

# Claude on Amazon Bedrock
claude_bedrock_agent = Agent(
    client=claude_bedrock_client,
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0.0,
    max_tokens=500,
    tools=tools,
    # planning=True, # as needed
)

bedrock_agent = Agent(
    client=bedrock_client,
    modelId="us.amazon.nova-pro-v1:0",
    tools=tools,
)

### Azure OpenAI

In [18]:
response = azure_openai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-14 10:02:08] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:08] INFO | root: Generating text
[2025-01-14 10:02:09] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_OmiA1IrFTjv3EOMiHLcKtyaR'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='call_E80ldh1h5CJYkYGQOHRJF8Ty')]
[2025-01-14 10:02:09] INFO | Running tool: start_music
[2025-01-14 10:02:09] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:09] INFO | Running tool: change_music_volume
[2025-01-14 10:02:09] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:10] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.8923318948344966, BPM: 120', error=None, call_id='call_OmiA1IrFTjv3EOMiHLcKtyaR', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='call_E80ldh1h5C

I've started playing some jazz music and set the volume to a low level. Enjoy!


In [19]:
response = await azure_openai_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-14 10:02:10] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:10] INFO | root: Generating text
[2025-01-14 10:02:12] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_BnsHxNq4DOAqgVYXxda2zzkD'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.3}', call_id='call_L9xjTel2uxJHrraCbsX7GDc7')]
[2025-01-14 10:02:12] INFO | Running tool: start_music
[2025-01-14 10:02:12] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:12] INFO | Running tool: change_music_volume
[2025-01-14 10:02:12] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:12] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.43890562180841464, BPM: 120', error=None, call_id='call_BnsHxNq4DOAqgVYXxda2zzkD', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.3', error=None, call_id='call_L9xjTel2u

I've started playing some jazz music and set the volume to a low level. Enjoy!


In [20]:
streamed_response = azure_openai_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:02:13] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:13] INFO | root: Generating text


{"ge
{"genre":
{"genre": "jazz
{"genre": "jazz"}
{"vo
{"volume"
{"volume": 0.3}


[2025-01-14 10:02:14] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_wusOorp3mOexS2iLB7rUenK3'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.3}', call_id='call_hL35BR7Ei0f6yJofmFf2XZbT')]
[2025-01-14 10:02:14] INFO | Running tool: start_music
[2025-01-14 10:02:14] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:14] INFO | Running tool: change_music_volume
[2025-01-14 10:02:14] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:14] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.7098992095096096, BPM: 120', error=None, call_id='call_wusOorp3mOexS2iLB7rUenK3', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.3', error=None, call_id='call_hL35BR7Ei0f6yJofmFf2XZbT', args='{"volume": 0.3}', name='change_music_volume')]
[2025-01-14 10:02:14] INFO | root: Generating text


I've
I've started
I've started playing
I've started playing some
I've started playing some jazz
I've started playing some jazz music
I've started playing some jazz music and
I've started playing some jazz music and set
I've started playing some jazz music and set the
I've started playing some jazz music and set the volume
I've started playing some jazz music and set the volume to
I've started playing some jazz music and set the volume to a
I've started playing some jazz music and set the volume to a low
I've started playing some jazz music and set the volume to a low level
I've started playing some jazz music and set the volume to a low level.
I've started playing some jazz music and set the volume to a low level. Enjoy
I've started playing some jazz music and set the volume to a low level. Enjoy!


[2025-01-14 10:02:17] DEBUG | Response: [TextResponse(text="I've started playing some jazz music and set the volume to a low level. Enjoy!")]


I've started playing some jazz music and set the volume to a low level. Enjoy!


In [21]:
streamed_response = azure_openai_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:02:17] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:17] INFO | root: Generating text


{"ge
{"genre":
{"genre": "jazz
{"genre": "jazz"}
{"vo
{"volume"
{"volume": 0.2}


[2025-01-14 10:02:19] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_tRd7Umsk1hZ5VqC61FdSfzO6'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='call_XQcvvQ9VWLGFjCpXV6zcmBAr')]
[2025-01-14 10:02:19] INFO | Running tool: start_music
[2025-01-14 10:02:19] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:19] INFO | Running tool: change_music_volume
[2025-01-14 10:02:19] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:19] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.8999064370188223, BPM: 120', error=None, call_id='call_tRd7Umsk1hZ5VqC61FdSfzO6', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='call_XQcvvQ9VWLGFjCpXV6zcmBAr', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-14 10:02:19] INFO | root: Generating text


I've
I've started
I've started playing
I've started playing some
I've started playing some jazz
I've started playing some jazz music
I've started playing some jazz music and
I've started playing some jazz music and set
I've started playing some jazz music and set the
I've started playing some jazz music and set the volume
I've started playing some jazz music and set the volume to
I've started playing some jazz music and set the volume to a
I've started playing some jazz music and set the volume to a low
I've started playing some jazz music and set the volume to a low level
I've started playing some jazz music and set the volume to a low level.
I've started playing some jazz music and set the volume to a low level. Enjoy
I've started playing some jazz music and set the volume to a low level. Enjoy!


[2025-01-14 10:02:20] DEBUG | Response: [TextResponse(text="I've started playing some jazz music and set the volume to a low level. Enjoy!")]


I've started playing some jazz music and set the volume to a low level. Enjoy!


### Gemini on Google AI Studio

In [22]:
response = google_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-14 10:02:20] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:20] INFO | root: Generating text
[2025-01-14 10:02:24] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='sg9zNvzBCayFxkMd6TbkY96c'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='KeYeS2KZjd5xppuogVeoVyZQ')]
[2025-01-14 10:02:24] INFO | Running tool: start_music
[2025-01-14 10:02:24] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:24] INFO | Running tool: change_music_volume
[2025-01-14 10:02:24] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:24] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.843756513071783, BPM: 120', error=None, call_id='sg9zNvzBCayFxkMd6TbkY96c', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='KeYeS2KZjd5xppuogVeoVyZQ', args

Ok, I've started playing jazz music at a low volume.


In [23]:
response = await google_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-14 10:02:25] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:25] INFO | root: Generating text
[2025-01-14 10:02:27] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='ZQsh4qz4GLNUnAqyjwl7OCmg'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='3dqQyDX3tqS34WoYaUZEWzmh')]
[2025-01-14 10:02:27] INFO | Running tool: start_music
[2025-01-14 10:02:27] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:27] INFO | Running tool: change_music_volume
[2025-01-14 10:02:27] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:27] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.1860416353482307, BPM: 120', error=None, call_id='ZQsh4qz4GLNUnAqyjwl7OCmg', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='3dqQyDX3tqS34WoYaUZEWzmh', arg

Ok, I've started playing jazz music at a low volume.


In [24]:
streamed_response = google_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:02:27] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:27] INFO | root: Generating text


{"genre": "jazz"}
{"volume": 0.2}


[2025-01-14 10:02:28] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='qsxho4juhsw4apK4y2BwuIE9'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='VZogPu6mR5P0UAlZZcoWZ296')]
[2025-01-14 10:02:28] INFO | Running tool: start_music
[2025-01-14 10:02:28] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:28] INFO | Running tool: change_music_volume
[2025-01-14 10:02:28] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:28] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.6776871594604403, BPM: 120', error=None, call_id='qsxho4juhsw4apK4y2BwuIE9', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='VZogPu6mR5P0UAlZZcoWZ296', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-14 10:02:28] INFO | root: Generating text


Ok
Ok, I've started playing jazz music at a low volume.



[2025-01-14 10:02:29] DEBUG | Response: [TextResponse(text="Ok, I've started playing jazz music at a low volume.\n")]


Ok, I've started playing jazz music at a low volume.



In [25]:
streamed_response = google_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:02:29] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:30] INFO | root: Generating text


{"genre": "jazz"}
{"volume": 0.2}


[2025-01-14 10:02:31] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='WeAq3g8urO9j5F4RJjzHf27i'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='cWwvXFw1PEsHd9ITc9oDMUIJ')]
[2025-01-14 10:02:31] INFO | Running tool: start_music
[2025-01-14 10:02:31] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:31] INFO | Running tool: change_music_volume
[2025-01-14 10:02:31] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:31] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.7812265611321745, BPM: 120', error=None, call_id='WeAq3g8urO9j5F4RJjzHf27i', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='cWwvXFw1PEsHd9ITc9oDMUIJ', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-14 10:02:31] INFO | root: Generating text


Ok
Ok, I've started playing jazz music at a low volume.



[2025-01-14 10:02:32] DEBUG | Response: [TextResponse(text="Ok, I've started playing jazz music at a low volume.\n")]


Ok, I've started playing jazz music at a low volume.



### Gemini on VertexAI

In [26]:
response = vertexai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-14 10:02:32] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:32] INFO | root: Generating text
[2025-01-14 10:02:34] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='rOn5omMCjITn8x8nzfSHXjJM'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='dgfRpVuLf3davIkOHjQM9zWh')]
[2025-01-14 10:02:34] INFO | Running tool: start_music
[2025-01-14 10:02:34] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:34] INFO | Running tool: change_music_volume
[2025-01-14 10:02:34] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:34] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.7433692415133876, BPM: 120', error=None, call_id='rOn5omMCjITn8x8nzfSHXjJM', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='dgfRpVuLf3davIkOHjQM9zWh', arg

Ok, I've started playing jazz music at a low volume.


In [27]:
response = await vertexai_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-14 10:02:35] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:35] INFO | root: Generating text
[2025-01-14 10:02:37] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='jqKAJkfEsDHorJlZVTwaQLtE'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='eFr4xlVfgvsyeu2dMaQqs7gI')]
[2025-01-14 10:02:37] INFO | Running tool: start_music
[2025-01-14 10:02:37] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:37] INFO | Running tool: change_music_volume
[2025-01-14 10:02:37] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:37] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.20524225038194654, BPM: 120', error=None, call_id='jqKAJkfEsDHorJlZVTwaQLtE', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='eFr4xlVfgvsyeu2dMaQqs7gI', ar

Ok, I've started playing jazz music at a low volume.


In [28]:
streamed_response = vertexai_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:02:37] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:37] INFO | root: Generating text


{"genre": "jazz"}
{"volume": 0.2}


[2025-01-14 10:02:38] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='nGKw0fVvqeVoUZtp1b8VDnVt'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='s4OTZnyfgMWtlsZv6rBj7gTe')]
[2025-01-14 10:02:38] INFO | Running tool: start_music
[2025-01-14 10:02:38] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:38] INFO | Running tool: change_music_volume
[2025-01-14 10:02:38] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:38] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.12904388048472804, BPM: 120', error=None, call_id='nGKw0fVvqeVoUZtp1b8VDnVt', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='s4OTZnyfgMWtlsZv6rBj7gTe', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-14 10:02:38] INFO | root: Generating text


Ok
Ok, I've started playing jazz music at a low volume.



[2025-01-14 10:02:40] DEBUG | Response: [TextResponse(text="Ok, I've started playing jazz music at a low volume.\n")]


Ok, I've started playing jazz music at a low volume.



In [29]:
streamed_response = vertexai_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:02:40] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:40] INFO | root: Generating text


{"genre": "jazz"}
{"volume": 0.2}


[2025-01-14 10:02:41] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='ArJta18nUWAJdbORG3mHv1PJ'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='CUkghtcwszfjjeLnbtls2Wgy')]
[2025-01-14 10:02:41] INFO | Running tool: start_music
[2025-01-14 10:02:41] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:41] INFO | Running tool: change_music_volume
[2025-01-14 10:02:41] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:41] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.732544960762771, BPM: 120', error=None, call_id='ArJta18nUWAJdbORG3mHv1PJ', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.2', error=None, call_id='CUkghtcwszfjjeLnbtls2Wgy', args='{"volume": 0.2}', name='change_music_volume')]
[2025-01-14 10:02:41] INFO | root: Generating text


Ok
Ok, I've started playing jazz music at a low volume.



[2025-01-14 10:02:43] DEBUG | Response: [TextResponse(text="Ok, I've started playing jazz music at a low volume.\n")]


Ok, I've started playing jazz music at a low volume.



### Claude on Anthropic

In [30]:
response = claude_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-14 10:02:43] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:43] INFO | root: Generating text
[2025-01-14 10:02:46] DEBUG | Response: [TextResponse(text="Certainly! I'd be happy to play some jazz music at a low volume for you. To do this, I'll need to use two functions: one to start the music and another to adjust the volume. Let's do that now."), ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='toolu_01EPNpZz4PpMBTbcxWXGgVbk'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='toolu_012RrSPtkSR6gdbSL7a7n6Fd')]
[2025-01-14 10:02:46] INFO | Running tool: start_music
[2025-01-14 10:02:46] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:46] INFO | Running tool: change_music_volume
[2025-01-14 10:02:46] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:46] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volum

Great! I've started playing some jazz music for you and adjusted the volume to a low level. Here's what I did:

1. Started playing jazz music using the "start_music" function.
2. Lowered the volume to 0.2 (on a scale from 0.0 to 1.0) using the "change_music_volume" function. This should give you a nice, soft background ambiance.

The music is now playing at a low volume. Is there anything else you'd like me to adjust with the music, such as changing the equalizer settings or switching to a different genre?


In [31]:
response = await claude_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-14 10:02:48] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:48] INFO | root: Generating text
[2025-01-14 10:02:51] DEBUG | Response: [TextResponse(text="Certainly! I'd be happy to play some jazz music at a low volume for you. To do this, I'll need to use two functions: one to start the music and another to adjust the volume. Let's do that now."), ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='toolu_01StZQhx1D4YHSatUAmMo1iH'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='toolu_01DvWJpyr13mNWu69dAuT4b4')]
[2025-01-14 10:02:51] INFO | Running tool: start_music
[2025-01-14 10:02:51] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:51] INFO | Running tool: change_music_volume
[2025-01-14 10:02:51] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:51] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volum

Great! I've started playing some jazz music for you and adjusted the volume to a low level. Here's what I did:

1. Started playing jazz music using the "start_music" function.
2. Lowered the volume to 0.2 (on a scale from 0.0 to 1.0) using the "change_music_volume" function. This should give you a nice, soft background ambiance.

The music is now playing at a low volume. Is there anything else you'd like me to adjust with the music, such as changing the equalizer settings or switching to a different genre?


In [32]:
streamed_response = claude_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:02:53] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:53] INFO | root: Generating text


Certainly! I'
Certainly! I'd be happy to play some jazz music at a low volume for you
Certainly! I'd be happy to play some jazz music at a low volume for you. To do this, I'll need to use two
Certainly! I'd be happy to play some jazz music at a low volume for you. To do this, I'll need to use two functions: one to start the music and another to adjust the volume. Let
Certainly! I'd be happy to play some jazz music at a low volume for you. To do this, I'll need to use two functions: one to start the music and another to adjust the volume. Let's do that now.
{"ge
{"genre": "
{"genre": "jaz
{"genre": "jazz"}
{"
{"volume
{"volume": 0.2}


[2025-01-14 10:02:56] DEBUG | Response: [TextResponse(text="Certainly! I'd be happy to play some jazz music at a low volume for you. To do this, I'll need to use two functions: one to start the music and another to adjust the volume. Let's do that now."), ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='toolu_017x45Nnoarg9rZ56vLoWGw3'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='toolu_013ghb6NWxGZ7pv7uwefySoZ')]
[2025-01-14 10:02:56] INFO | Running tool: start_music
[2025-01-14 10:02:56] INFO | Tool: start_music successfully ran.
[2025-01-14 10:02:56] INFO | Running tool: change_music_volume
[2025-01-14 10:02:56] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:02:56] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.6361428741878828, BPM: 120', error=None, call_id='toolu_017x45Nnoarg9rZ56vLoWGw3', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to

Great! I've
Great! I've started playing some jazz music and adjusted the volume to a low
Great! I've started playing some jazz music and adjusted the volume to a low level for you. Here's what I di
Great! I've started playing some jazz music and adjusted the volume to a low level for you. Here's what I did:

1. Started playing jazz music using
Great! I've started playing some jazz music and adjusted the volume to a low level for you. Here's what I did:

1. Started playing jazz music using the "start_music" function.
2
Great! I've started playing some jazz music and adjusted the volume to a low level for you. Here's what I did:

1. Started playing jazz music using the "start_music" function.
2. Lowered the volume to
Great! I've started playing some jazz music and adjusted the volume to a low level for you. Here's what I did:

1. Started playing jazz music using the "start_music" function.
2. Lowered the volume to 0.2 (on a scale from 0
Great! I've started playing some jazz music and adj

[2025-01-14 10:02:59] DEBUG | Response: [TextResponse(text='Great! I\'ve started playing some jazz music and adjusted the volume to a low level for you. Here\'s what I did:\n\n1. Started playing jazz music using the "start_music" function.\n2. Lowered the volume to 0.2 (on a scale from 0.0 to 1.0) using the "change_music_volume" function. This should be quite low and suitable for background listening.\n\nThe music is now playing at a low volume. Is there anything else you\'d like me to adjust with the music, such as changing the equalizer settings or switching to a different genre?')]


Great! I've started playing some jazz music and adjusted the volume to a low level for you. Here's what I did:

1. Started playing jazz music using the "start_music" function.
2. Lowered the volume to 0.2 (on a scale from 0.0 to 1.0) using the "change_music_volume" function. This should be quite low and suitable for background listening.

The music is now playing at a low volume. Is there anything else you'd like me to adjust with the music, such as changing the equalizer settings or switching to a different genre?


In [33]:
streamed_response = claude_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:02:59] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:02:59] INFO | root: Generating text


Certainly! I'
Certainly! I'd be happy to play some jazz music at a low volume for you
Certainly! I'd be happy to play some jazz music at a low volume for you. To do this, I'll need to use two
Certainly! I'd be happy to play some jazz music at a low volume for you. To do this, I'll need to use two functions: one to start the music and another to
Certainly! I'd be happy to play some jazz music at a low volume for you. To do this, I'll need to use two functions: one to start the music and another to adjust the volume. Let's do that now.
{"gen
{"genre":
{"genre": "ja
{"genre": "jazz"}
{"volu
{"volume
{"volume": 0.2}


[2025-01-14 10:03:01] DEBUG | Response: [TextResponse(text="Certainly! I'd be happy to play some jazz music at a low volume for you. To do this, I'll need to use two functions: one to start the music and another to adjust the volume. Let's do that now."), ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='toolu_01H2DJK38NgUxhjCXGR3ABt4'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='toolu_01Aqt9gorLZsidWB16Y54FNU')]
[2025-01-14 10:03:01] INFO | Running tool: start_music
[2025-01-14 10:03:01] INFO | Tool: start_music successfully ran.
[2025-01-14 10:03:01] INFO | Running tool: change_music_volume
[2025-01-14 10:03:01] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:03:01] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.13495664621554326, BPM: 120', error=None, call_id='toolu_01H2DJK38NgUxhjCXGR3ABt4', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume t

Great! I've
Great! I've started playing some jazz music for you and adjuste
Great! I've started playing some jazz music for you and adjusted the volume to a low level. Here's what I di
Great! I've started playing some jazz music for you and adjusted the volume to a low level. Here's what I did:

1. Started playing jazz music using the "start_music
Great! I've started playing some jazz music for you and adjusted the volume to a low level. Here's what I did:

1. Started playing jazz music using the "start_music" function.
2. Lowered the volume to
Great! I've started playing some jazz music for you and adjusted the volume to a low level. Here's what I did:

1. Started playing jazz music using the "start_music" function.
2. Lowered the volume to 0.2 (on a scale from 0
Great! I've started playing some jazz music for you and adjusted the volume to a low level. Here's what I did:

1. Started playing jazz music using the "start_music" function.
2. Lowered the volume to 0.2 (on a scale from 0.0

[2025-01-14 10:03:04] DEBUG | Response: [TextResponse(text='Great! I\'ve started playing some jazz music for you and adjusted the volume to a low level. Here\'s what I did:\n\n1. Started playing jazz music using the "start_music" function.\n2. Lowered the volume to 0.2 (on a scale from 0.0 to 1.0) using the "change_music_volume" function. This should give you a nice, soft background ambiance.\n\nThe music is now playing at a low volume. Is there anything else you\'d like me to adjust with the music, such as changing the equalizer settings or switching to a different genre?')]


Great! I've started playing some jazz music for you and adjusted the volume to a low level. Here's what I did:

1. Started playing jazz music using the "start_music" function.
2. Lowered the volume to 0.2 (on a scale from 0.0 to 1.0) using the "change_music_volume" function. This should give you a nice, soft background ambiance.

The music is now playing at a low volume. Is there anything else you'd like me to adjust with the music, such as changing the equalizer settings or switching to a different genre?


### Amazon Bedrock Converse API model

In [34]:
response = bedrock_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-14 10:03:04] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:03:04] INFO | root: Generating text
[2025-01-14 10:03:06] DEBUG | Response: [TextResponse(text='<thinking> The user has requested to play jazz music at a low volume. I should first start playing jazz music and then adjust the volume to a low setting. </thinking>\n'), ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='tooluse__WfTNnt_R9uJAmW5kdzodA'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.3}', call_id='tooluse_mz5FNFFpReWrS-wHEKFtsA')]
[2025-01-14 10:03:06] INFO | Running tool: start_music
[2025-01-14 10:03:06] INFO | Tool: start_music successfully ran.
[2025-01-14 10:03:06] INFO | Running tool: change_music_volume
[2025-01-14 10:03:06] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:03:06] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.15753816039112578, 

<thinking> The jazz music has started playing and the volume has been adjusted to 0.3, which is a low volume setting. I should inform the user that their request has been completed. </thinking> I have started playing jazz music and adjusted the volume to a low setting as you requested. Enjoy the music!


In [35]:
streamed_response = bedrock_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-14 10:03:07] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:03:07] INFO | root: Generating text


<thinking
<thinking>
<thinking> The
<thinking> The user
<thinking> The user has
<thinking> The user has requested
<thinking> The user has requested to
<thinking> The user has requested to play
<thinking> The user has requested to play jazz
<thinking> The user has requested to play jazz music
<thinking> The user has requested to play jazz music at
<thinking> The user has requested to play jazz music at a
<thinking> The user has requested to play jazz music at a low
<thinking> The user has requested to play jazz music at a low volume
<thinking> The user has requested to play jazz music at a low volume.
<thinking> The user has requested to play jazz music at a low volume. I
<thinking> The user has requested to play jazz music at a low volume. I should
<thinking> The user has requested to play jazz music at a low volume. I should first
<thinking> The user has requested to play jazz music at a low volume. I should first start
<thinking> The user has requested to play jazz music at a low vol

[2025-01-14 10:03:09] DEBUG | Response: [TextResponse(text='<thinking> The user has requested to play jazz music at a low volume. I should first start playing jazz music and then adjust the volume to a low level. </thinking>\n'), ToolCallResponse(name='start_music', args='{"genre":"jazz"}', call_id='tooluse_p8JtJcSQTBiPGTjX7QByCg'), ToolCallResponse(name='change_music_volume', args='{"volume":0.3}', call_id='tooluse_HXeIQTNaSR2WDYtQFhyywA')]
[2025-01-14 10:03:09] INFO | Running tool: start_music
[2025-01-14 10:03:09] INFO | Tool: start_music successfully ran.
[2025-01-14 10:03:09] INFO | Running tool: change_music_volume
[2025-01-14 10:03:09] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:03:09] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.6926396892923625, BPM: 120', error=None, call_id='tooluse_p8JtJcSQTBiPGTjX7QByCg', args='{"genre":"jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.3', error=None, call_id='t

<thinking
<thinking>
<thinking> The
<thinking> The tool
<thinking> The tool results
<thinking> The tool results indicate
<thinking> The tool results indicate that
<thinking> The tool results indicate that jazz
<thinking> The tool results indicate that jazz music
<thinking> The tool results indicate that jazz music has
<thinking> The tool results indicate that jazz music has been
<thinking> The tool results indicate that jazz music has been started
<thinking> The tool results indicate that jazz music has been started and
<thinking> The tool results indicate that jazz music has been started and the
<thinking> The tool results indicate that jazz music has been started and the volume
<thinking> The tool results indicate that jazz music has been started and the volume has
<thinking> The tool results indicate that jazz music has been started and the volume has been
<thinking> The tool results indicate that jazz music has been started and the volume has been changed
<thinking> The tool result

[2025-01-14 10:03:10] DEBUG | Response: [TextResponse(text='<thinking> The tool results indicate that jazz music has been started and the volume has been changed to 0.3. I should inform the user that their request has been fulfilled. </thinking> The jazz music has been started and the volume has been adjusted to a low level as per your request. Enjoy the music!')]


<thinking> The tool results indicate that jazz music has been started and the volume has been changed to 0.3. I should inform the user that their request has been fulfilled. </thinking> The jazz music has been started and the volume has been adjusted to a low level as per your request. Enjoy the music!


NOTE : Bedrock client currently doesn't support asynchronous generation.

# Structured output using tool calling

## Response schema as a tool

Structured output in langrila is just tool calling (please refer to [01.introduction.ipynb](./01.introduction.ipynb)), so you can get structured response even if you are using tool calling.

In [36]:
from pydantic import BaseModel, Field


class RessponseSchema(BaseModel):
    genre: MusicGenre = Field(
        ...,
        description="The genre of music.",
    )
    bpm: int = Field(
        ...,
        description="The beats per minute of the music.",
        ge=50,
        le=200,
    )
    volume: float = Field(
        ...,
        description="The volume level of the music.",
        ge=0,
        le=1,
    )
    equalizer_settings: EqualizerSettings = Field(
        ...,
        description="The equalizer settings.",
    )

In [37]:
openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
    # planning=True,
    response_schema_as_tool=RessponseSchema,
)

In [38]:
response = openai_agent.generate_text(prompt)

RessponseSchema.model_validate_json(response.contents[0].text)

[2025-01-14 10:03:10] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:03:10] INFO | root: Generating text
[2025-01-14 10:03:11] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre":"jazz"}', call_id='call_fo9rIvtcAJhZJZonpwhmpVF1')]
[2025-01-14 10:03:11] INFO | Running tool: start_music
[2025-01-14 10:03:11] INFO | Tool: start_music successfully ran.
[2025-01-14 10:03:11] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.7396531309585548, BPM: 120', error=None, call_id='call_fo9rIvtcAJhZJZonpwhmpVF1', args='{"genre":"jazz"}', name='start_music')]
[2025-01-14 10:03:11] INFO | root: Generating text
[2025-01-14 10:03:12] DEBUG | Response: [ToolCallResponse(name='change_music_volume', args='{"volume":0.2}', call_id='call_M1K4ZsRqpAZzUorAwsyvGem9')]
[2025-01-14 10:03:12] INFO | Running tool: change_music_volume
[2025-01-14 10:03:12] INFO | Tool: change_music_volume successfully

RessponseSchema(genre=<MusicGenre.jazz: 'jazz'>, bpm=120, volume=0.2, equalizer_settings=EqualizerSettings(hz_100=0, hz_200=0, hz_400=0, hz_800=0, hz_1600=0, hz_3200=0, hz_6400=0))

Same for other clients, so examples will be omitted.

## Native response schema

If you want to use native structured output, you can specify the original parameters of LLM provider API. For example, for OpenAI API, you can use `response_format` parameter supported by OpenAI API natively. Please be aware of limitations of the API in this case.

In [39]:
from pydantic import BaseModel, Field


# default value and range limitation are not supported by OpenAI API's response format
class NativeEqualizerSettings(BaseModel):
    hz_100: int = Field(
        ...,
        description="The 100 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_200: int = Field(
        ...,
        description="The 200 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_400: int = Field(
        ...,
        description="The 400 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_800: int = Field(
        ...,
        description="The 800 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_1600: int = Field(
        ...,
        description="The 1600 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_3200: int = Field(
        ...,
        description="The 3200 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )
    hz_6400: int = Field(
        ...,
        description="The 4000 Hz band in dB. It should be between -12 and 12. Default is 0.",
    )


class NativeResponseSchema(BaseModel):
    genre: MusicGenre = Field(..., description="The genre of music to play.")
    bpm: int = Field(
        ...,
        description="The BPM of the music.",
    )
    volume: float = Field(
        ...,
        description="The volume level to set the music to. It should be between 0.0 and 1.0.",
    )
    equalizer_settings: NativeEqualizerSettings = Field(
        ..., description="The equalizer settings to set."
    )

Re-definition to use native response format.

In [40]:
def change_equalizer_settings_native(settings: NativeEqualizerSettings) -> str:
    """
    Change the equalizer settings of the music.

    Parameters
    ----------
    settings : NativeEqualizerSettings
        The new equalizer settings to set.

    Returns
    ----------
    str
        A message indicating that the equalizer settings have been changed.
    """
    return f"Changing equalizer settings to {settings.model_dump()}"

In [41]:
tools_native = [start_music, change_music_volume, change_music, change_equalizer_settings_native]

In [42]:
openai_agent_native = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=tools_native,
    response_format=NativeResponseSchema,
)

In [43]:
response = openai_agent_native.generate_text(prompt)

NativeResponseSchema.model_validate_json(response.contents[0].text)

[2025-01-14 10:03:14] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:03:14] INFO | root: Generating text
[2025-01-14 10:03:15] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_i1CTL37edgAhUeJu1XVq9eIK'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.3}', call_id='call_UQqLCxj3YcK6UoeBnZ7ttxfJ')]
[2025-01-14 10:03:15] INFO | Running tool: start_music
[2025-01-14 10:03:15] INFO | Tool: start_music successfully ran.
[2025-01-14 10:03:15] INFO | Running tool: change_music_volume
[2025-01-14 10:03:15] INFO | Tool: change_music_volume successfully ran.
[2025-01-14 10:03:15] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: jazz, Volume: 0.9277869537693915, BPM: 120', error=None, call_id='call_i1CTL37edgAhUeJu1XVq9eIK', args='{"genre": "jazz"}', name='start_music'), ToolUsePrompt(output='Changing volume to 0.3', error=None, call_id='call_UQqLCxj3Yc

NativeResponseSchema(genre=<MusicGenre.jazz: 'jazz'>, bpm=120, volume=0.3, equalizer_settings=NativeEqualizerSettings(hz_100=0, hz_200=0, hz_400=0, hz_800=0, hz_1600=0, hz_3200=0, hz_6400=0))

# Dependency injection

While tools can be passed to an Agent directly as functions, using the Tool class allows for more flexible control. One example of this flexibility is dependency injection. 

Standard tool calling requires the LLM to generate all arguments specified as 'required'. For arguments not specified as 'required', either a default value or a value generated by the LLM is used.

With the Tool class, you can use a 'context' argument to separate parameters into those generated by the LLM and those injected from an external source. This notebook demonstrates an example of this approach.

Here is an example of tool invokes an other agent. 

In [44]:
def run_music_agent(
    agent: Agent,  # type: ignore
    agent_name: str,
    instruction: str,
) -> str:
    """
    This function is used to run the agent which has the capabilities to manage the music settings.
    Agent can use the following tools:

    Tools:
    start_music: Turn on the music. The genre, BPM, and volume are randomly selected.
    change_bpm: Change the BPM of the music.
    change_music: Change the music genre and BPM.

    Parameters
    ----------
    agent : Agent
        The agent instance.
    agent_name : str
        The name of the agent.
    instruction : str
        The detail and specific instruction to the agent, including the plan to get answer.

    Returns
    ----------
    str
        The response from the agent.
    """
    if not isinstance(agent, Agent):
        raise ValueError(
            "Subagent must be an instance of Agent class. "
            "Please provide the correct agent instance."
        )

    return agent.generate_text(instruction, name=agent_name).contents[0].text  # type: ignore

To show tha example of dependency injection, two agents are defined; `music_agent` for managing music setting, and another agent calls `music_agent`.

In [45]:
# The agent for managing music settings
music_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=[start_music, change_music_volume, change_music, change_equalizer_settings],
)

# The main agent to call the music agent
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=[
        Tool(tool=run_music_agent, context={"agent": music_agent, "agent_name": "music_agent"}),
    ],
)

The tool's parameters specified in the context arguments of Tool instance are injected externally. In this case, `agent` argument and `agent_name` argument of the run_music_agent tool are injected.

In [46]:
agent.tools[0].context

{'agent': Agent(name=root), 'agent_name': 'music_agent'}

In [47]:
response = agent.generate_text(prompt)

[2025-01-14 10:03:16] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:03:16] INFO | root: Generating text
[2025-01-14 10:03:17] DEBUG | Response: [ToolCallResponse(name='run_music_agent', args='{"instruction":"Start playing jazz music at a low volume."}', call_id='call_ioySokB23DqRoE6aRQ0xADJP')]
[2025-01-14 10:03:17] INFO | Running tool: run_music_agent
[2025-01-14 10:03:17] DEBUG | Prompt: [TextPrompt(text='Start playing jazz music at a low volume.')]
[2025-01-14 10:03:17] INFO | music_agent: Generating text
[2025-01-14 10:03:18] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_pxq5aDtoFC73DYTaBk5JceN1'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='call_zyRwdhZMOKBlSmtUjjOucnSR')]
[2025-01-14 10:03:18] INFO | Running tool: start_music
[2025-01-14 10:03:18] INFO | Tool: start_music successfully ran.
[2025-01-14 10:03:18] INFO | Running t

As you can see in the log messages above, the agent instance invoked music_agent, but the parameters created by the agent to call `music_agent` is only `instruction` argument while `agent` argument and `agent_name` are injected. The arguments are also validated when the tool is called. If you passed wrong value for a context parameter, the error will occur when run tool.

In [48]:
from langrila import AgentConfig

# Define not to retry the request even if an error occurs
agent_config = AgentConfig(
    max_error_retries=1,
)

# The main agent to call the music agent
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=[
        Tool(
            tool=run_music_agent,
            # wrong for agent argument below
            context={"agent": "wrong_value", "agent_name": "music_agent"},
        ),
    ],
    agent_config=agent_config,
)

In [49]:
response = agent.generate_text(prompt)

[2025-01-14 10:03:19] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:03:19] INFO | root: Generating text
[2025-01-14 10:03:20] DEBUG | Response: [ToolCallResponse(name='run_music_agent', args='{"instruction":"Start playing jazz music at a low volume."}', call_id='call_FLHzRCpKH7WY6qA4TLTM1mjR')]
[2025-01-14 10:03:20] INFO | Running tool: run_music_agent
[2025-01-14 10:03:20] ERROR | Validation error occurred while running tool. Retrying...
Tool: run_music_agent
Attribute: agent
Input value: wrong_value
ValidationError: Input should be an instance of Agent


RetryLimitExceededError: Retry limit exceeded. Please try again or change the request parameters.

Actually the tools calling other agent such as this are dynamically created in the Agent class if you pass the subagents parameter of the agent, but in this notebook, dared to define a tool as an example and passed it to the Agent class as a tool. For multi-agent example, Please refer to [03.multi_agent.ipynb](./03.multi_agent.ipynb)

# Tool result serializer

Tool class accepts `serializer` parameter that is callable object to convert any value to string. `str` is used in default, but you can also use custom serializer. This capability allows us to reuse the tools implemented in daily development by only implementing serializer. Let's change the example above using custom serializer.

In [50]:
import json

from langrila import Response


def run_music_agent(
    agent: Agent,  # type: ignore
    agent_name: str,
    instruction: str,
) -> Response:
    """
    This function is used to run the agent which has the capabilities to manage the music settings.
    Agent can use the following tools:

    Tools:
    start_music: Turn on the music. The genre, BPM, and volume are randomly selected.
    change_bpm: Change the BPM of the music.
    change_music: Change the music genre and BPM.

    Parameters
    ----------
    agent : Agent
        The agent instance.
    agent_name : str
        The name of the agent.
    instruction : str
        The detail and specific instruction to the agent, including the plan to get answer.

    Returns
    ----------
    Response
        The response from the agent.
    """
    if not isinstance(agent, Agent):
        raise ValueError(
            "Subagent must be an instance of Agent class. "
            "Please provide the correct agent instance."
        )

    return agent.generate_text(instruction, name=agent_name)


def response_serializer(response: Response) -> str:
    """
    Serialize the response to a string.

    Parameters
    ----------
    response : Response
        The response from the agent.

    Returns
    ----------
    str
        The serialized response.
    """
    return json.dumps(response.model_dump(include={"role", "contents", "name"}), ensure_ascii=False)

In [51]:
from langrila import AgentConfig

# Define not to retry the request even if an error occurs
agent_config = AgentConfig(
    max_error_retries=1,
)

# The main agent to call the music agent
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=[
        Tool(
            tool=run_music_agent,
            context={"agent": music_agent, "agent_name": "music_agent"},
            serializer=response_serializer,  # serializer
        ),
    ],
)

In [52]:
response = agent.generate_text(prompt)

[2025-01-14 10:04:09] DEBUG | Prompt: [TextPrompt(text='Play some music, well, play jazz music and prefer to low volume.')]
[2025-01-14 10:04:09] INFO | root: Generating text
[2025-01-14 10:04:10] DEBUG | Response: [ToolCallResponse(name='run_music_agent', args='{"instruction":"Start playing jazz music at a low volume."}', call_id='call_0l2pcXp0JUUZ3QMzy0JL7lpa')]
[2025-01-14 10:04:10] INFO | Running tool: run_music_agent
[2025-01-14 10:04:10] DEBUG | Prompt: [TextPrompt(text='Start playing jazz music at a low volume.')]
[2025-01-14 10:04:10] INFO | music_agent: Generating text
[2025-01-14 10:04:11] DEBUG | Response: [ToolCallResponse(name='start_music', args='{"genre": "jazz"}', call_id='call_R0JG56AonabQPWTszdvi7qXU'), ToolCallResponse(name='change_music_volume', args='{"volume": 0.2}', call_id='call_2uuKTaQ547tmfw1sh0AEqG4L')]
[2025-01-14 10:04:11] INFO | Running tool: start_music
[2025-01-14 10:04:11] INFO | Tool: start_music successfully ran.
[2025-01-14 10:04:11] INFO | Running t

ToolUsePrompt of the `run_music_agent` in the log message is changed to json string as expected.